<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Evaluate machine learning models</h1>
<h3 align="right">Written by the KSO Team</h3>

# Set up KSO requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso-object-detection and its requirements</font> { vertical-output: true }
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
    !pip install -q --upgrade pip
    !pip install -qr koster_yolov4/requirements.txt
    !pip install -qr koster_yolov4/yolov5_tracker/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Replace upsampling script with custom version
    os.chdir("koster_yolov4/tutorials")
    !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/multi_tracker_zoo.py

    # Add detection script for FRCNN model
    !cp ../src/frcnn_detect.py ../yolov5/frcnn_detect.py

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    print("All packages are installed and ready to go!")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys
    import pkgutil

    if pkgutil.find_loader("torch") is None:
        !pip install -q --upgrade pip
        !pip install -q torch==1.8.0 torchvision==0.12.0

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/multi_tracker_zoo.py
    # Add detection script for FRCNN model
    !cp ../src/frcnn_detect.py ../yolov5/frcnn_detect.py
    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    clear_output()
    print("Running locally... you're good to go!")

#######Import Python packages########

# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
sys.path.append("..")


# Set to display dataframes as interactive tables
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.server_utils as s_utils
import kso_utils.t6_utils as t6
import kso_utils.t5_utils as t5
import torch
import wandb

clear_output()
print("Packages loaded successfully")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Select the model type for evaluation</font> { vertical-output: true }
model_type = t5.choose_model_type()

In [ ]:
# Model-specific imports
if model_type.value == 1:
    import yolov5.detect as detect

    print("Object detection model loaded")
elif model_type.value == 2:
    import yolov5.classify.predict as detect

    print("Image classification model loaded")
elif model_type.value == 3:
    import yolov5.segment.predict as detect

    print("Image segmentation model loaded")
elif model_type.value == 4:
    import yolov5.frcnn_detect as detect

    print("Faster RCNN model loaded")
else:
    print("Invalid model specification")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose your project</font> { vertical-output: true }
project_name = t_utils.choose_project()

In [ ]:
team_name = t6.get_team_name(project_name.value)

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

# Evaluate model on custom footage

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose the model</font> { vertical-output: true }
model = t6.choose_model(project_name.value, team_name=team_name)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder to download the model</font> { vertical-output: true }
download_dir = t_utils.choose_folder(".", "where to download the model")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Download model</font> { vertical-output: true }
artifact_dir = t6.get_model(
    model_name=model.value,
    project_name=project_name.value,
    download_path=download_dir.selected,
    team_name=team_name,
)

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Choose your custom footage</font> { vertical-output: true }
source = t_utils.choose_footage(
    project,
    project.movie_folder if project.movie_folder is not None else ".",
    "custom footage",
)

In [ ]:
source_value = t_utils.process_source(source)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder where to save the runs</font> { vertical-output: true }
# This should be left as default value in most cases.
save_dir = t_utils.choose_folder(".", "runs output")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Start the run in Weights and Biases</font> { vertical-output: true }
run = wandb.init(
    entity=team_name,
    project="model-evaluations",
    settings=wandb.Settings(start_method="fork"),
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose a confidence threshold for evaluation</font> { vertical-output: true }
conf_thres = t6.choose_conf()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Run model over selected custom footage</font> { vertical-output: true }
detect.run(
    weights=[
        f
        for f in Path(artifact_dir).iterdir()
        if f.is_file() and str(f).endswith((".pt", ".model")) and "osnet" not in str(f)
    ][0],
    source=source_value,
    conf_thres=conf_thres.value,
    save_txt=True,
    save_conf=True,
    project=save_dir.selected,
    name="detect",
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder with the evaluation data</font> { vertical-output: true }
eval_dir = t_utils.choose_folder(
    save_dir.selected if "save_dir" in vars() else ".", "runs output"
)

In [ ]:
config = t6.set_config(conf_thres.value, model.value, eval_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Add the data to Weights and Biases</font> { vertical-output: true }
t6.add_data_wandb(eval_dir.selected, "detection_output", run)

In [ ]:
# @title <font size="5">↓ ឵឵<i>View model output</font> { vertical-output: true }
viewer = t6.choose_files(eval_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Investigate training and validation datasets (only image data)</font> { vertical-output: true }
train_dataset, val_dataset = t6.get_dataset(project_name.value, model.value)

In [ ]:
# Training set
t6.get_data_viewer(os.path.join(train_dataset, "data/images"))

In [ ]:
# Validation set
t6.get_data_viewer(os.path.join(val_dataset, "data/images"))

# Track unique individuals (Optional)

In [ ]:
latest_tracker = t6.track_objects(
    source_dir=source_value,
    artifact_dir=artifact_dir,
    tracker_folder=eval_dir.selected,
    conf_thres=conf_thres.value,
    img_size=(540, 540),
    gpu=True if torch.cuda.is_available() else False,
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Add the data to Weights and Biases</font> { vertical-output: true }
t6.add_data_wandb(Path(latest_tracker).parent.absolute(), "tracker_output", run)

# Generate reports

In [ ]:
# @title <font size="5">↓ ឵឵<i>Generate classification report and counts by species</font> { vertical-output: true }
# It requires Step 2
csv_report = t6.generate_csv_report(eval_dir.selected, wandb_log=True)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Generate tracking report and counts by species</font> { vertical-output: true }
# It requires Step 3
tracking_report = t6.generate_counts(
    eval_dir.selected, latest_tracker, artifact_dir, wandb_log=True
)

🔴 <span style="color:red">&nbsp;NOTE: Run this cell to complete WANDB run, OR else artifacts will not be shown.

In [ ]:
wandb.finish()

In [ ]:
# END